Nhiệm vụ 2: truy vấn và thống kê mô tả

In [1]:
#Đọc file
import pandas as pd
import numpy as np

citizen_df = pd.read_csv("../data_clean/citizen_info_clean.csv")
feedback_df = pd.read_csv("../data_clean/feedback_log_clean.csv")
resolution_df = pd.read_csv("../data_clean/resolution_time_clean.csv")
# đảm bảo datetime
feedback_df['submit_time'] = pd.to_datetime(feedback_df['submit_time'])
resolution_df['resolved_time'] = pd.to_datetime(resolution_df['resolved_time'])


In [2]:
#2.1Thống kê dữ liệu cư dân area
citizen_by_area = (
    citizen_df
    .groupby('area')['citizen_id']
    .nunique()
    .reset_index(name='citizen_count')
    .sort_values('citizen_count', ascending=False)
)

citizen_by_area


,area,citizen_count
0,phuong a,13
2,quan 1,11
3,qu·∫≠n 1,8
4,xa b,6
1,ph∆∞·ªùng a,4
5,xa b,4
6,x√£ b,4


In [3]:
#2.2Thống kê số lượng phản ánh theo category
feedback_by_category = (
    feedback_df
    .groupby('category')
    .size()
    .reset_index(name='feedback_count')
    .sort_values('feedback_count', ascending=False)
)

feedback_by_category

,category,feedback_count
3,ha tang,19
6,m√¥i tr∆∞·ªùng,16
7,thu tuc,14
0,an ninh,13
2,ha tang,13
5,moi truong,12
4,h·∫° t·∫ßng,12
8,th·ªß t·ª•c h√†nh ch√≠nh,11
1,an ninh,10


In [4]:
#2.3Gộp feedback và citizen để lấy area
feedback_area = feedback_df.merge(
    citizen_df[['citizen_id', 'area']],
    on='citizen_id',
    how='left'
)


In [5]:
#group by channel+area
feedback_by_channel_area = (
    feedback_area
    .groupby(['area', 'channel'])
    .size()
    .reset_index(name='feedback_count')
    .sort_values(['area', 'feedback_count'], ascending=[True, False])
)

feedback_by_channel_area


,area,channel,feedback_count
0,phuong a,app,4
1,phuong a,hot line,3
4,phuong a,website,2
2,phuong a,hotline,1
3,phuong a,tr·ª±c ti·∫øp,1
7,ph∆∞·ªùng a,website,2
5,ph∆∞·ªùng a,app,1
6,ph∆∞·ªùng a,web site,1
8,quan 1,app,7
10,quan 1,hotline,4


In [6]:
#2.4Tìm phản ánh thông tin chưa xử lý
feedback_resolution = feedback_df.merge(
    resolution_df[['feedback_id', 'status']],
    on='feedback_id',
    how='left'
)

unresolved_feedback = feedback_resolution[
    feedback_resolution['status'].isna() |
    (feedback_resolution['status'].str.strip() == '')
]

unresolved_feedback

,feedback_id,citizen_id,category,channel,department,submit_time,status
2,FB1001,CD114,ha tang,web site,phong tnmt,NaT,NaN
3,FB1002,CD142,ha tang,hotline,ubnd phuong,2024-01-30 15:36:00,NaN
4,FB1003,cd102,m√¥i tr∆∞·ªùng,hot line,cong an,NaT,NaN
10,FB1007,CD127,ha tang,website,cong an,2024-02-28 12:23:00,NaN
13,FB1009,cd147,thu tuc,hotline,phong tnmt,NaT,NaN
...,...,...,...,...,...,...,...
133,FB1106,CD112,thu tuc,web site,cong an,2024-02-17 04:57:00,NaN
139,FB1110,CD123,an ninh,hotline,phong tnmt,NaT,NaN
145,FB1114,CD140,an ninh,app,ubnd phuong,2024-01-14 10:20:00,NaN
147,FB1115,CD148,ha tang,hot line,ubnd phuong,NaT,NaN


In [8]:
# 2.5 Thống kê số phản ánh tồn đọng/quá hạn

THRESHOLD_HOURS = 72  #ngưỡng số giờ xử lý do giảng viên quy định
#phản ánh tồn đọng
backlog_feedback = feedback_resolution.merge(
    resolution_df[['feedback_id', 'resolution_hours']],
    on='feedback_id',
    how='left'
)
backlog_feedback = backlog_feedback[
    backlog_feedback['resolution_hours'].isna() |
    (backlog_feedback['resolution_hours'] > THRESHOLD_HOURS)
]

backlog_feedback
#thống kê số lượng tồn đọng
backlog_count = backlog_feedback.shape[0]
backlog_count



121